<a href="https://colab.research.google.com/github/khushbooekhande/DrugRecommendationSystem/blob/master/DiseasePrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import csv
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 
from sklearn.linear_model import LogisticRegression

In [290]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
data = pd.read_excel('/content/drive/My Drive/New_Dataset/Symptoms_dataset.xlsx')

In [292]:
data

,Disease,Count of Disease Occurrence,Symptom
0,UMLS:C0020538_hypertensive disease,3363.0,UMLS:C0008031_pain chest
1,NaN,NaN,UMLS:C0392680_shortness of breath
2,NaN,NaN,UMLS:C0012833_dizziness
3,NaN,NaN,UMLS:C0004093_asthenia
4,NaN,NaN,UMLS:C0085639_fall
...,...,...,...
1861,NaN,NaN,UMLS:C0425251_bedridden^UMLS:C0741453_bedridden
1862,NaN,NaN,UMLS:C0242453_prostatism
1863,UMLS:C0011127_decubitus ulcer,42.0,UMLS:C0232257_systolic murmur
1864,NaN,NaN,UMLS:C0871754_frail


In [293]:
data = data.fillna(method='ffill')
data.shape

(1866, 3)

In [0]:
# Clean Disease and Symptom Names
def clean_data(data):
    data_list = []
    data_name = data.replace('^','_').split('_')
    n = 1
    for names in data_name:
        if (n % 2 == 0):
            data_list.append(names)
        n += 1
    return data_list

In [0]:
disease_list = []
disease_symptom_dict = defaultdict(list)
disease_symptom_count = {}
count = 0

for idx, row in data.iterrows():
    
    # Get the Disease Names
    if (row['Disease'] !="\xc2\xa0") and (row['Disease'] != ""):
        disease = row['Disease']
        disease_list = clean_data(data=disease)
        count = row['Count of Disease Occurrence']

    # Get the Symptoms Corresponding to Diseases
    if (row['Symptom'] !="\xc2\xa0") and (row['Symptom'] != ""):
        symptom = row['Symptom']
        symptom_list = clean_data(data=symptom)
        for d in disease_list:
            for s in symptom_list:
                disease_symptom_dict[d].append(s)
            disease_symptom_count[d] = count

In [0]:
# Writing the data in a csv file

with open("/content/drive/My Drive/Symptoms_clean.csv","w") as csvfile:
    writer = csv.writer(csvfile)
    for key,values in disease_symptom_dict.items():
        for v in values:
            #key = str.encode(key)
            key = str.encode(key).decode('utf-8')
            #.strip()
            #v = v.encode('utf-8').strip()
            #v = str.encode(v)
            writer.writerow([key,v,disease_symptom_count[key]])

In [0]:
columns = ['Disease', 'Symptoms', 'Count']
data_clean = pd.read_csv("/content/drive/My Drive/Symptoms_clean.csv", names = columns, encoding ="ISO-8859-1")

In [298]:
data_clean

,Disease,Symptoms,Count
0,hypertensive disease,pain chest,3363.0
1,hypertensive disease,shortness of breath,3363.0
2,hypertensive disease,dizziness,3363.0
3,hypertensive disease,asthenia,3363.0
4,hypertensive disease,fall,3363.0
...,...,...,...
2125,affect labile,bedridden,45.0
2126,affect labile,prostatism,45.0
2127,decubitus ulcer,systolic murmur,42.0
2128,decubitus ulcer,frail,42.0


In [299]:
len(data_clean["Disease"].unique())

149

In [300]:
len(data_clean["Symptoms"].unique())

405

In [301]:
df = pd.DataFrame(data_clean)
df

,Disease,Symptoms,Count
0,hypertensive disease,pain chest,3363.0
1,hypertensive disease,shortness of breath,3363.0
2,hypertensive disease,dizziness,3363.0
3,hypertensive disease,asthenia,3363.0
4,hypertensive disease,fall,3363.0
...,...,...,...
2125,affect labile,bedridden,45.0
2126,affect labile,prostatism,45.0
2127,decubitus ulcer,systolic murmur,42.0
2128,decubitus ulcer,frail,42.0


In [302]:
df_dc = df[['Disease', 'Count']]
df_dc

,Disease,Count
0,hypertensive disease,3363.0
1,hypertensive disease,3363.0
2,hypertensive disease,3363.0
3,hypertensive disease,3363.0
4,hypertensive disease,3363.0
...,...,...
2125,affect labile,45.0
2126,affect labile,45.0
2127,decubitus ulcer,42.0
2128,decubitus ulcer,42.0


In [303]:
df_dc.sort_values(by=['Disease'], inplace=True)
df_dc

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Disease,Count
1442,Alzheimer's disease,101.0
1452,Alzheimer's disease,101.0
1459,Alzheimer's disease,101.0
1458,Alzheimer's disease,101.0
1457,Alzheimer's disease,101.0
...,...,...
1190,upper respiratory infection,135.0
1189,upper respiratory infection,135.0
1188,upper respiratory infection,135.0
1186,upper respiratory infection,135.0


In [304]:
df_dc=df_dc.drop_duplicates('Disease')
df_dc

,Disease,Count
1442,Alzheimer's disease,101.0
377,HIV,350.0
1361,PneumocystisÂ cariniiÂ pneumonia,113.0
119,accidentÂ cerebrovascular,885.0
361,acquiredÂ immuno-deficiency syndrome,350.0
...,...,...
1612,tonic-clonic seizures,92.0
842,transient ischemic attack,168.0
1432,tricuspid valve insufficiency,101.0
1103,ulcer peptic,143.0


In [305]:
df_s = df_dc['Disease']
df_s = pd.DataFrame(df_s)
df_s

,Disease
1442,Alzheimer's disease
377,HIV
1361,PneumocystisÂ cariniiÂ pneumonia
119,accidentÂ cerebrovascular
361,acquiredÂ immuno-deficiency syndrome
...,...
1612,tonic-clonic seizures
842,transient ischemic attack
1432,tricuspid valve insufficiency
1103,ulcer peptic


In [306]:
df_c = df_dc['Count']
df_c = pd.DataFrame(df_c)
df_c

,Count
1442,101.0
377,350.0
1361,113.0
119,885.0
361,350.0
...,...
1612,92.0
842,168.0
1432,101.0
1103,143.0


In [307]:
df_1 = pd.get_dummies(df.Symptoms)
df_1

,Heberden's node,Murphy's sign,Stahli's line,abdomen acute,abdominal bloating,abdominal tenderness,abnormal sensation,abnormally hard consistency,abortion,abscess bacterial,absences finding,achalasia,ache,adverse effect,adverse reaction,agitation,air fluid level,alcohol binge episode,alcoholic withdrawal symptoms,ambidexterity,angina pectoris,anorexia,anosmia,aphagia,apyrexial,arthralgia,ascites,asterixis,asthenia,asymptomatic,ataxia,atypia,aura,awakening early,barking cough,bedridden,behavior hyperactive,behavior showing increased motor activity,blackout,blanch,...,tenesmus,terrify,thicken,throat sore,throbbing sensation quality,tinnitus,tired,titubation,todd paralysis,tonic seizures,transaminitis,transsexual,tremor,tremor resting,tumor cell invasion,unable to concentrate,unconscious state,uncoordination,underweight,unhappy,unresponsiveness,unsteady gait,unwell,urge incontinence,urgency ofÂ micturition,urinary hesitation,urinoma,verbal auditory hallucinations,verbally abusive behavior,vertigo,vision blurred,vomiting,weepiness,weight gain,welt,wheelchair bound,wheezing,withdraw,worry,yellow sputum
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2125,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2126,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2127,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2128,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [308]:
df_pivoted = pd.concat([df_s,df_1], axis=1)
df_pivoted

,Disease,Heberden's node,Murphy's sign,Stahli's line,abdomen acute,abdominal bloating,abdominal tenderness,abnormal sensation,abnormally hard consistency,abortion,abscess bacterial,absences finding,achalasia,ache,adverse effect,adverse reaction,agitation,air fluid level,alcohol binge episode,alcoholic withdrawal symptoms,ambidexterity,angina pectoris,anorexia,anosmia,aphagia,apyrexial,arthralgia,ascites,asterixis,asthenia,asymptomatic,ataxia,atypia,aura,awakening early,barking cough,bedridden,behavior hyperactive,behavior showing increased motor activity,blackout,...,tenesmus,terrify,thicken,throat sore,throbbing sensation quality,tinnitus,tired,titubation,todd paralysis,tonic seizures,transaminitis,transsexual,tremor,tremor resting,tumor cell invasion,unable to concentrate,unconscious state,uncoordination,underweight,unhappy,unresponsiveness,unsteady gait,unwell,urge incontinence,urgency ofÂ micturition,urinary hesitation,urinoma,verbal auditory hallucinations,verbally abusive behavior,vertigo,vision blurred,vomiting,weepiness,weight gain,welt,wheelchair bound,wheezing,withdraw,worry,yellow sputum
0,hypertensive disease,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2125,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2126,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2127,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2128,decubitus ulcer,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [309]:
df_pivoted = df_pivoted.groupby('Disease').sum()
df_pivoted

,Heberden's node,Murphy's sign,Stahli's line,abdomen acute,abdominal bloating,abdominal tenderness,abnormal sensation,abnormally hard consistency,abortion,abscess bacterial,absences finding,achalasia,ache,adverse effect,adverse reaction,agitation,air fluid level,alcohol binge episode,alcoholic withdrawal symptoms,ambidexterity,angina pectoris,anorexia,anosmia,aphagia,apyrexial,arthralgia,ascites,asterixis,asthenia,asymptomatic,ataxia,atypia,aura,awakening early,barking cough,bedridden,behavior hyperactive,behavior showing increased motor activity,blackout,blanch,...,tenesmus,terrify,thicken,throat sore,throbbing sensation quality,tinnitus,tired,titubation,todd paralysis,tonic seizures,transaminitis,transsexual,tremor,tremor resting,tumor cell invasion,unable to concentrate,unconscious state,uncoordination,underweight,unhappy,unresponsiveness,unsteady gait,unwell,urge incontinence,urgency ofÂ micturition,urinary hesitation,urinoma,verbal auditory hallucinations,verbally abusive behavior,vertigo,vision blurred,vomiting,weepiness,weight gain,welt,wheelchair bound,wheezing,withdraw,worry,yellow sputum
Disease,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Alzheimer's disease,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
HIV,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
PneumocystisÂ cariniiÂ pneumonia,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
accidentÂ cerebrovascular,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
acquiredÂ immuno-deficiency syndrome,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tonic-clonic seizures,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
transient ischemic attack,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
tricuspid valve insufficiency,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [310]:
data = df_pivoted.join(df_dc.set_index('Disease'), on='Disease')
data

,Heberden's node,Murphy's sign,Stahli's line,abdomen acute,abdominal bloating,abdominal tenderness,abnormal sensation,abnormally hard consistency,abortion,abscess bacterial,absences finding,achalasia,ache,adverse effect,adverse reaction,agitation,air fluid level,alcohol binge episode,alcoholic withdrawal symptoms,ambidexterity,angina pectoris,anorexia,anosmia,aphagia,apyrexial,arthralgia,ascites,asterixis,asthenia,asymptomatic,ataxia,atypia,aura,awakening early,barking cough,bedridden,behavior hyperactive,behavior showing increased motor activity,blackout,blanch,...,terrify,thicken,throat sore,throbbing sensation quality,tinnitus,tired,titubation,todd paralysis,tonic seizures,transaminitis,transsexual,tremor,tremor resting,tumor cell invasion,unable to concentrate,unconscious state,uncoordination,underweight,unhappy,unresponsiveness,unsteady gait,unwell,urge incontinence,urgency ofÂ micturition,urinary hesitation,urinoma,verbal auditory hallucinations,verbally abusive behavior,vertigo,vision blurred,vomiting,weepiness,weight gain,welt,wheelchair bound,wheezing,withdraw,worry,yellow sputum,Count
Disease,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Alzheimer's disease,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,101.0
HIV,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,350.0
PneumocystisÂ cariniiÂ pneumonia,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,113.0
accidentÂ cerebrovascular,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,885.0
acquiredÂ immuno-deficiency syndrome,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,350.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tonic-clonic seizures,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,92.0
transient ischemic attack,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,168.0
tricuspid valve insufficiency,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,101.0


# **METHOD** **1**

Disease prediction using the original dataset without considering the disease occurance count

In [311]:
cols = data.columns
cols = cols[1:-1]
cols

Index(['Murphy's sign', 'Stahli's line', 'abdomen acute', 'abdominal bloating',
       'abdominal tenderness', 'abnormal sensation',
       'abnormally hard consistency', 'abortion', 'abscess bacterial',
       'absences finding',
       ...
       'vision blurred', 'vomiting', 'weepiness', 'weight gain', 'welt',
       'wheelchair bound', 'wheezing', 'withdraw', 'worry', 'yellow sputum'],
      dtype='object', length=403)

In [0]:
X = df_new[cols]
y = df_new['Disease']

In [0]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = y)

Multibinomial Naive bayes

In [314]:
mnb = MultinomialNB()
mnb = mnb.fit(x_train, y_train)
y_pred = mnb.predict(x_test)
print("Accuracy of the MNB model in Predicting Disease: ", accuracy_score(y_test, y_pred)*100, '%')

Accuracy of the MNB model in Predicting Disease:  80.3038105046344 %


ExtraTree Classifier

In [315]:
ext = ExtraTreesClassifier(n_estimators=100, random_state=0)
ext = ext.fit(x_train, y_train)
y_pred = ext.predict(x_test)
print("Accuracy of the ExtraTree classifier model in Predicting Disease: ", accuracy_score(y_test, y_pred)*100, '%')

Accuracy of the ExtraTree classifier model in Predicting Disease:  80.3038105046344 %


Decision tree classifier

In [316]:
for i in range(10, 111, 20):
  dtc = DecisionTreeClassifier(max_depth=i)
  dtc.fit(x_train,y_train)
  y_pred = dtc.predict(x_test)
  print("Accuracy of the ExtraTree classifier model in Predicting Disease: ", accuracy_score(y_test, y_pred)*100, '%')

Accuracy of the ExtraTree classifier model in Predicting Disease:  37.33264675592173 %
Accuracy of the ExtraTree classifier model in Predicting Disease:  57.36354273944387 %
Accuracy of the ExtraTree classifier model in Predicting Disease:  68.03553038105046 %
Accuracy of the ExtraTree classifier model in Predicting Disease:  74.9356333676622 %
Accuracy of the ExtraTree classifier model in Predicting Disease:  79.3898043254377 %
Accuracy of the ExtraTree classifier model in Predicting Disease:  80.3038105046344 %


### Disease prediction using method 1

In [0]:
importances = dtc.feature_importances_
indices = np.argsort(importances)[::-1]

features = cols

In [318]:
#Getting the top 10 symptoms.
for f in range(10):
    print("%d. feature %d - %s (%f)" % (f + 1, indices[f], features[indices[f]] ,importances[indices[f]]))

1. feature 256 - pain chest (0.110955)
2. feature 355 - sweating increased (0.049266)
3. feature 199 - irritable mood (0.046382)
4. feature 37 - blackout (0.046280)
5. feature 19 - angina pectoris (0.044393)
6. feature 57 - chest tightness (0.044279)
7. feature 110 - facial paresis (0.030901)
8. feature 58 - chill (0.026227)
9. feature 84 - distress respiratory (0.025887)
10. feature 327 - shortness of breath (0.024839)


In [0]:
feature_dict = {}
for i,f in enumerate(features):
    feature_dict[f] = i

In [320]:
feature_dict['shortness of breath']
symptomDR = [i/332 if i == 332 else i*0 for i in range(len(features))]
print(len(symptomDR))
symptomDR = np.array(symptomDR).reshape(1,len(symptomDR))

403


In [321]:
dtc.predict(symptomDR)

array(['neuropathy'], dtype=object)

In [322]:
ext.predict(symptomDR)

array(['neuropathy'], dtype=object)

In [323]:
mnb.predict(symptomDR)

array(['neuropathy'], dtype='<U36')

Predicting based on 2 features

In [0]:
input_features = [feature_dict['distress respiratory'], feature_dict['angina pectoris']]
input_features
arr =[]
for i in range(len(features)):
  if i == input_features[0]:
    i = int(i/input_features[0])
  elif i == input_features[1]:
    i = int(i/input_features[1])
  else:
    i = 0
  arr.append(i)
arr = np.array(arr).reshape(-1,len(arr))

In [325]:
dtc.predict(arr)

array(['coronary arteriosclerosis'], dtype=object)

In [326]:
ext.predict(arr)

array(['coronary arteriosclerosis'], dtype=object)

In [327]:
mnb.predict(arr)

array(['coronary arteriosclerosis'], dtype='<U36')

predicting taking three features

In [0]:
input_features = [feature_dict['distress respiratory'], feature_dict['irritable mood'],  feature_dict['irritable mood']]
input_features
arr =[]
for i in range(len(features)):
  if i == input_features[0]:
    i = int(i/input_features[0])
  elif i == input_features[1]:
    i = int(i/input_features[1])
  elif i == input_features[2]:
    i = int(i/input_features[2])
  else:
    i = 0
  arr.append(i)
arr = np.array(arr).reshape(-1,len(arr))

In [329]:
dtc.predict(arr)

array(['depression mental'], dtype=object)

In [330]:
ext.predict(arr)

array(['depression mental'], dtype=object)

In [331]:
mnb.predict(symptomDR)

array(['neuropathy'], dtype='<U36')

# METHOD 2

In [332]:
data =data.reset_index()
data

,Disease,Heberden's node,Murphy's sign,Stahli's line,abdomen acute,abdominal bloating,abdominal tenderness,abnormal sensation,abnormally hard consistency,abortion,abscess bacterial,absences finding,achalasia,ache,adverse effect,adverse reaction,agitation,air fluid level,alcohol binge episode,alcoholic withdrawal symptoms,ambidexterity,angina pectoris,anorexia,anosmia,aphagia,apyrexial,arthralgia,ascites,asterixis,asthenia,asymptomatic,ataxia,atypia,aura,awakening early,barking cough,bedridden,behavior hyperactive,behavior showing increased motor activity,blackout,...,terrify,thicken,throat sore,throbbing sensation quality,tinnitus,tired,titubation,todd paralysis,tonic seizures,transaminitis,transsexual,tremor,tremor resting,tumor cell invasion,unable to concentrate,unconscious state,uncoordination,underweight,unhappy,unresponsiveness,unsteady gait,unwell,urge incontinence,urgency ofÂ micturition,urinary hesitation,urinoma,verbal auditory hallucinations,verbally abusive behavior,vertigo,vision blurred,vomiting,weepiness,weight gain,welt,wheelchair bound,wheezing,withdraw,worry,yellow sputum,Count
0,Alzheimer's disease,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,101.0
1,HIV,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,350.0
2,PneumocystisÂ cariniiÂ pneumonia,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,113.0
3,accidentÂ cerebrovascular,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,885.0
4,acquiredÂ immuno-deficiency syndrome,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,350.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,tonic-clonic seizures,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,92.0
145,transient ischemic attack,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,168.0
146,tricuspid valve insufficiency,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,101.0
147,ulcer peptic,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,143.0


Multiplying the rows

In [0]:
data['Count'] = data['Count'].astype(int)

In [334]:
data.dtypes

Disease            object
Heberden's node     uint8
Murphy's sign       uint8
Stahli's line       uint8
abdomen acute       uint8
                    ...  
wheezing            uint8
withdraw            uint8
worry               uint8
yellow sputum       uint8
Count               int64
Length: 406, dtype: object

In [0]:
df_new = pd.DataFrame([data.iloc[idx] 
                       for idx in data.index 
                       for i in range(data.iloc[idx]['Count'])]).reset_index(drop=True)

In [336]:
df_new.shape

(38839, 406)

In [337]:
df_new.head()

,Disease,Heberden's node,Murphy's sign,Stahli's line,abdomen acute,abdominal bloating,abdominal tenderness,abnormal sensation,abnormally hard consistency,abortion,abscess bacterial,absences finding,achalasia,ache,adverse effect,adverse reaction,agitation,air fluid level,alcohol binge episode,alcoholic withdrawal symptoms,ambidexterity,angina pectoris,anorexia,anosmia,aphagia,apyrexial,arthralgia,ascites,asterixis,asthenia,asymptomatic,ataxia,atypia,aura,awakening early,barking cough,bedridden,behavior hyperactive,behavior showing increased motor activity,blackout,...,terrify,thicken,throat sore,throbbing sensation quality,tinnitus,tired,titubation,todd paralysis,tonic seizures,transaminitis,transsexual,tremor,tremor resting,tumor cell invasion,unable to concentrate,unconscious state,uncoordination,underweight,unhappy,unresponsiveness,unsteady gait,unwell,urge incontinence,urgency ofÂ micturition,urinary hesitation,urinoma,verbal auditory hallucinations,verbally abusive behavior,vertigo,vision blurred,vomiting,weepiness,weight gain,welt,wheelchair bound,wheezing,withdraw,worry,yellow sputum,Count
0,Alzheimer's disease,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,101
1,Alzheimer's disease,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,101
2,Alzheimer's disease,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,101
3,Alzheimer's disease,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,101
4,Alzheimer's disease,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,101


In [338]:
cols = df_new.columns
cols = cols[1:-1]
cols

Index(['Heberden's node', 'Murphy's sign', 'Stahli's line', 'abdomen acute',
       'abdominal bloating', 'abdominal tenderness', 'abnormal sensation',
       'abnormally hard consistency', 'abortion', 'abscess bacterial',
       ...
       'vision blurred', 'vomiting', 'weepiness', 'weight gain', 'welt',
       'wheelchair bound', 'wheezing', 'withdraw', 'worry', 'yellow sputum'],
      dtype='object', length=404)

In [0]:
X = df_new[cols]
y = df_new['Disease']

In [0]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = y)

In [341]:
x_train.shape

(31071, 404)

Multibinomial Naive Bayes 

In [342]:
mnb = MultinomialNB()
mnb = mnb.fit(x_train, y_train)
y_pred = mnb.predict(x_test)
print("Accuracy of the MNB model in Predicting Disease: ", accuracy_score(y_test, y_pred)*100, '%')

Accuracy of the MNB model in Predicting Disease:  80.3038105046344 %


Extratree Classifier

In [343]:
ext = ExtraTreesClassifier(n_estimators=100, random_state=0)
ext = ext.fit(x_train, y_train)
y_pred = ext.predict(x_test)
print("Accuracy of the ExtraTree classifier model in Predicting Disease: ", accuracy_score(y_test, y_pred)*100, '%')

Accuracy of the ExtraTree classifier model in Predicting Disease:  80.3038105046344 %


Decision Tree Classifier

In [344]:
for i in range(10, 111, 20):
  dtc = DecisionTreeClassifier(max_depth=i)
  dtc.fit(x_train,y_train)
  y_pred = dtc.predict(x_test)
  print("Accuracy of the ExtraTree classifier model in Predicting Disease: ", accuracy_score(y_test, y_pred)*100, '%')
  

Accuracy of the ExtraTree classifier model in Predicting Disease:  37.33264675592173 %
Accuracy of the ExtraTree classifier model in Predicting Disease:  57.36354273944387 %
Accuracy of the ExtraTree classifier model in Predicting Disease:  68.03553038105046 %
Accuracy of the ExtraTree classifier model in Predicting Disease:  74.9356333676622 %
Accuracy of the ExtraTree classifier model in Predicting Disease:  79.3898043254377 %
Accuracy of the ExtraTree classifier model in Predicting Disease:  80.3038105046344 %


Disease Prediction using method two

In [0]:
importances = dtc.feature_importances_
indices = np.argsort(importances)[::-1]

features = cols

In [346]:
#Getting the top 10 symptoms.
for f in range(10):
    print("%d. feature %d - %s (%f)" % (f + 1, indices[f], features[indices[f]] ,importances[indices[f]]))

1. feature 257 - pain chest (0.110955)
2. feature 356 - sweating increased (0.049266)
3. feature 200 - irritable mood (0.046382)
4. feature 38 - blackout (0.046280)
5. feature 58 - chest tightness (0.044393)
6. feature 20 - angina pectoris (0.044279)
7. feature 111 - facial paresis (0.030901)
8. feature 59 - chill (0.026227)
9. feature 85 - distress respiratory (0.025887)
10. feature 328 - shortness of breath (0.024839)


In [0]:
feature_dict = {}
for i,f in enumerate(features):
    feature_dict[f] = i

Disease prediction using 1 feature

In [0]:
feature_dict['shortness of breath']
symptomDR = [i/332 if i == 332 else i*0 for i in range(len(features))]
len(symptomDR)
symptomDR = np.array(symptomDR).reshape(1,len(symptomDR))

In [349]:
dtc.predict(symptomDR)

array(['decubitus ulcer'], dtype=object)

In [350]:
ext.predict(symptomDR)

array(['fibroid tumor'], dtype=object)

In [351]:
mnb.predict(symptomDR)

array(['hypertensive disease'], dtype='<U36')

Disease prediction using features

In [0]:
input_features = [feature_dict['distress respiratory'], feature_dict['angina pectoris']]
input_features
arr =[]
for i in range(len(features)):
  if i == input_features[0]:
    i = int(i/input_features[0])
  elif i == input_features[1]:
    i = int(i/input_features[1])
  else:
    i = 0
  arr.append(i)
arr = np.array(arr).reshape(-1,len(arr))

In [353]:
dtc.predict(arr)

array(['coronary arteriosclerosis'], dtype=object)

In [354]:
ext.predict(arr)

array(['coronary arteriosclerosis'], dtype=object)

In [355]:
mnb.predict(arr)

array(['coronary arteriosclerosis'], dtype='<U36')

In [0]:
input_features = [feature_dict['distress respiratory'], feature_dict['irritable mood'],  feature_dict['irritable mood']]
input_features
arr =[]
for i in range(len(features)):
  if i == input_features[0]:
    i = int(i/input_features[0])
  elif i == input_features[1]:
    i = int(i/input_features[1])
  elif i == input_features[2]:
    i = int(i/input_features[2])
  else:
    i = 0
  arr.append(i)
arr = np.array(arr).reshape(-1,len(arr))

In [357]:
dtc.predict(arr)

array(['depression mental'], dtype=object)

In [358]:
ext.predict(arr)

array(['depression mental'], dtype=object)

In [359]:
mnb.predict(arr)

array(['depression mental'], dtype='<U36')